<a href="https://colab.research.google.com/github/daa103/DAA_ESAA/blob/main/%ED%95%AD%EA%B3%B5%ED%8E%B8%EC%98%88%EC%B8%A1_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
plt.rc('font', family='NanumBarunGothic') #matplotlib 한글 깨짐 방지
%matplotlib inline

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
train=pd.read_csv("/content/drive/MyDrive/ESAA_data/train.csv")
test=pd.read_csv("/content/drive/MyDrive/ESAA_data/test.csv")
submission=pd.read_csv("/content/drive/MyDrive/ESAA_data/sample_submission.csv")

###**EDA(데이터탐색)**
* ID : 샘플 고유 id
* 항공편 운항 관련 정보
* Month: 해당 항공편의 출발 월
* Day_of_Month: Month에 해당하는 월의 날짜
* Estimated_Departure_Time: 전산 시스템을 바탕으로 측정된 비행기의 출발 시간 (현지 시각, HH:MM 형식)
* Estimated_Arrival_Time: 전산 시스템을 바탕으로 측정된 비행기의 도착 시간 (현지 시각, HH:MM 형식)
* Cancelled: 해당 항공편의 취소 여부 (0: 취소되지 않음, 1: 취소됨)
* Diverted: 해당 항공편의 경유 여부 (0: 취소되지 않음, 1: 취소됨)
* Origin_Airport: 해당 항공편 출발 공항의 고유 코드 (IATA 공항 코드)
* Origin_Airport_ID: 해당 항공편 출발 공항의 고유 ID (US DOT ID)
* Origin_State: 해당 항공편 출발 공항이 위치한 주의 이름
* Destination_Airport: 해당 항공편 도착 공항의 고유 코드 (IATA 공항 코드)
* Destination_Airport_ID: 해당 항공편 도착 공항의 고유 ID (US DOT ID)
* Destination_State: 해당 항공편 도착 공항이 위치한 주의 이름
* Distance: 출발 공항과 도착 공항 사이의 거리 (mile 단위)
* Airline: 해당 항공편을 운항하는 항공사
* Carrier_Code(IATA): 해당 항공편을 운항하는 항공사의 고유 코드
(IATA 공항 코드, 단 다른 항공사가 같은 코드를 보유할 수도 있음)
* Carrier_ID(DOT): 해당 항공편을 운항하는 항공사의 고유 ID (US DOT ID)
* Tail_Number: 해당 항공편을 운항하는 항공기의 고유 등록번호
* Delay: 항공편 지연 여부 (Not_Delayed, Delayed)
예측해야 하는 타깃
>
다수의 데이터에 레이블이 존재하지 않음
>
준지도학습을 통해 레이블링 가능

**데이터구성**

In [18]:
train.head()

,ID,Month,Day_of_Month,Estimated_Departure_Time,Estimated_Arrival_Time,Cancelled,Diverted,Origin_Airport,Origin_Airport_ID,Origin_State,Destination_Airport,Destination_Airport_ID,Destination_State,Distance,Airline,Carrier_Code(IATA),Carrier_ID(DOT),Tail_Number,Delay
5,TRAIN_000005,4,13,1545.0,NaN,0,0,EWR,11618,NaN,DCA,11278,Virginia,199.0,Republic Airlines,UA,20452.0,N657RW,Not_Delayed
6,TRAIN_000006,1,20,1742.0,1903.0,0,0,EWR,11618,New Jersey,BOS,10721,Massachusetts,200.0,United Air Lines Inc.,UA,NaN,N66825,Not_Delayed
8,TRAIN_000008,6,13,1420.0,1550.0,0,0,BWI,10821,NaN,CLT,11057,North Carolina,361.0,Southwest Airlines Co.,WN,19393.0,N765SW,Not_Delayed
10,TRAIN_000010,8,13,1730.0,1844.0,0,0,DCA,11278,Virginia,PIT,14122,Pennsylvania,204.0,Republic Airlines,AA,NaN,N119HQ,Delayed
12,TRAIN_000012,1,12,1015.0,1145.0,0,0,CLE,11042,Ohio,DEN,11292,Colorado,1201.0,Southwest Airlines Co.,WN,NaN,N8696E,Not_Delayed


In [19]:
test.head()

,ID,Month,Day_of_Month,Estimated_Departure_Time,Estimated_Arrival_Time,Cancelled,Diverted,Origin_Airport,Origin_Airport_ID,Origin_State,Destination_Airport,Destination_Airport_ID,Destination_State,Distance,Airline,Carrier_Code(IATA),Carrier_ID(DOT),Tail_Number
0,TEST_000000,12,16,1156.0,NaN,0,0,IAH,12266,Texas,SAT,14683,Texas,191.0,United Air Lines Inc.,UA,NaN,N79402
1,TEST_000001,9,12,1500.0,1715.0,0,0,EWR,11618,New Jersey,ATL,10397,NaN,746.0,Delta Air Lines Inc.,DL,19790.0,N3765
2,TEST_000002,3,6,1600.0,1915.0,0,0,ORD,13930,Illinois,LGA,12953,New York,733.0,United Air Lines Inc.,UA,19977.0,N413UA
3,TEST_000003,5,18,1920.0,2045.0,0,0,OAK,13796,California,LAX,12892,California,337.0,Southwest Airlines Co.,WN,19393.0,N905WN
4,TEST_000004,7,7,1915.0,2152.0,0,0,FLL,11697,Florida,LAX,12892,California,2343.0,JetBlue Airways,B6,20409.0,N945JT


In [20]:
submission.head()

,ID,Not_Delayed,Delayed
0,TEST_000000,0,1
1,TEST_000001,0,1
2,TEST_000002,0,1
3,TEST_000003,0,1
4,TEST_000004,0,1


In [21]:
print(train.shape)
print(test.shape)
print(submission.shape)

(255001, 19)
(1000000, 18)
(1000000, 3)


**데이터요약**

In [22]:
train.info()
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 255001 entries, 5 to 999992
Data columns (total 19 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   ID                        255001 non-null  object 
 1   Month                     255001 non-null  int64  
 2   Day_of_Month              255001 non-null  int64  
 3   Estimated_Departure_Time  227160 non-null  float64
 4   Estimated_Arrival_Time    227317 non-null  float64
 5   Cancelled                 255001 non-null  int64  
 6   Diverted                  255001 non-null  int64  
 7   Origin_Airport            255001 non-null  object 
 8   Origin_Airport_ID         255001 non-null  int64  
 9   Origin_State              227145 non-null  object 
 10  Destination_Airport       255001 non-null  object 
 11  Destination_Airport_ID    255001 non-null  int64  
 12  Destination_State         227323 non-null  object 
 13  Distance                  255001 non-null  f

In [23]:
pd.options.display.float_format = '{:.1f}'.format # 소숫점 자리수 출력값 format 설정

**요약통계량**

In [24]:
train.describe()

,Month,Day_of_Month,Estimated_Departure_Time,Estimated_Arrival_Time,Cancelled,Diverted,Origin_Airport_ID,Destination_Airport_ID,Distance,Carrier_ID(DOT)
count,255001.0,255001.0,227160.0,227317.0,255001.0,255001.0,255001.0,255001.0,255001.0,227231.0
mean,7.0,15.8,1319.4,1472.6,0.0,0.0,12697.6,12700.0,779.9,19995.6
std,3.5,8.8,490.9,517.5,0.0,0.0,1516.0,1515.9,589.3,403.7
min,1.0,1.0,1.0,1.0,0.0,0.0,10135.0,10135.0,16.0,19393.0
25%,4.0,8.0,905.0,1053.0,0.0,0.0,11292.0,11292.0,349.0,19790.0
50%,7.0,16.0,1310.0,1500.0,0.0,0.0,12889.0,12889.0,620.0,19977.0
75%,10.0,23.0,1730.0,1910.0,0.0,0.0,14057.0,14057.0,1014.0,20378.0
max,12.0,31.0,2359.0,2400.0,0.0,0.0,16869.0,16869.0,5095.0,21171.0


In [25]:
test.describe()

,Month,Day_of_Month,Estimated_Departure_Time,Estimated_Arrival_Time,Cancelled,Diverted,Origin_Airport_ID,Destination_Airport_ID,Distance,Carrier_ID(DOT)
count,1000000.0,1000000.0,891016.0,890952.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,890994.0
mean,6.7,15.6,1377.1,1522.0,0.0,0.0,12688.0,12699.6,795.8,20001.3
std,3.5,8.7,487.6,528.3,0.0,0.0,1509.3,1512.9,594.6,402.9
min,1.0,1.0,1.0,1.0,0.0,0.0,10135.0,10135.0,16.0,19393.0
25%,3.0,8.0,954.8,1127.0,0.0,0.0,11292.0,11292.0,358.0,19790.0
50%,7.0,16.0,1405.0,1555.0,0.0,0.0,12889.0,12889.0,634.0,19977.0
75%,10.0,23.0,1800.0,1944.0,0.0,0.0,14027.0,14057.0,1034.0,20378.0
max,12.0,31.0,2400.0,2400.0,0.0,0.0,16869.0,16869.0,5095.0,21171.0


**결측치확인**

In [26]:
train.isna().sum()

ID                              0
Month                           0
Day_of_Month                    0
Estimated_Departure_Time    27841
Estimated_Arrival_Time      27684
Cancelled                       0
Diverted                        0
Origin_Airport                  0
Origin_Airport_ID               0
Origin_State                27856
Destination_Airport             0
Destination_Airport_ID          0
Destination_State           27678
Distance                        0
Airline                     27540
Carrier_Code(IATA)          27888
Carrier_ID(DOT)             27770
Tail_Number                     0
Delay                           0
dtype: int64

In [27]:
test.isna().sum()

ID                               0
Month                            0
Day_of_Month                     0
Estimated_Departure_Time    108984
Estimated_Arrival_Time      109048
Cancelled                        0
Diverted                         0
Origin_Airport                   0
Origin_Airport_ID                0
Origin_State                106505
Destination_Airport              0
Destination_Airport_ID           0
Destination_State           106523
Distance                         0
Airline                     106527
Carrier_Code(IATA)          108993
Carrier_ID(DOT)             109006
Tail_Number                      0
dtype: int64

결측값 존재 Column
* Estimated_Departure_Time: 전산 시스템을 바탕으로 측정된 비행기의 출발 시간
* Estimated_Arrival_Time: 전산 시스템을 바탕으로 측정된 비행기의 도착 시간
* Origin_State: 해당 항공편 출발 공항이 위치한 주의 이름
* Destination_State: 해당 항공편 도착 공항이 위치한 주의 이름
* Airline: 해당 항공편을 운항하는 항공사
* Carrier_Code(IATA): 해당 항공편을 운항하는 항공사의 고유 코드
* Carrier_ID(DOT): 해당 항공편을 운항하는 항공사의 고유 ID (US DOT ID)
* **Delay: 항공편 지연 여부 (Not_Delayed, Delayed) 예측해야 하는 타깃**

In [ ]:
# ID, 취소항공편, 경유항공편, 항공사고유코드 칼럼 제거(항공사 고유 ID칼럼과 중복정보)
train= train.drop(columns=['ID','Cancelled','Diverted','Carrier_Code(IATA)'],axis=1)
test= test.drop(columns=['ID','Cancelled','Diverted','Carrier_Code(IATA)'],axis=1)